# Concrete Cracking

In [2]:
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.1")
import sys; sys.path.append('../') # Locate the helpers file
from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)
Helpers.delete_all_database_contents(lusas.database())
db = lusas.database()
db.createAnalysisStructural("Analysis 1")
db.setAnalysisCategory("2D Inplane")


Inputs

In [3]:
wall_height = 2
wall_thickness = 0.3
beam_length = 4.5
beam_thickness = 0.45
base_width = 1.4
base_thickness = 0.5
cover = 0.05

In [4]:
class Point:
    def __init__(self, x:float, y:float):
        self.X = x
        self.Y = y
        self.LusasPoint = None
    
    def create_lusas_point(self):
        # geometryData object contains all the settings to perform a geometry creation
        geom_data = lusas.geometryData().setAllDefaults().setLowerOrderGeometryType("coordinates")
        geom_data.addCoords(self.X, self.Y, 0)
        self.LusasPoint = lusas.database().createPoint(geom_data).getObjects("Point")[0]

    def offset(self, x:float, y:float) -> 'Point':
        return Point(self.X + x, self.Y + y)

class Line:
    def __init__(self, p1:Point, p2:Point):
        self.P1 = p1
        self.P2 = p2
        self.LusasLine = None

    def create_lusas_line(self):
        # Create the points in lusas first if they have not alraedy been created
        if self.P1.LusasPoint is None : self.P1.create_lusas_point()
        if self.P2.LusasPoint is None : self.P2.create_lusas_point()

        # geometryData object contains all the settings to perform a geometry creation
        geom_data = lusas.geometryData().setAllDefaults()         
        # set the options for creating straight lines from points
        geom_data.setCreateMethod("straight").setLowerOrderGeometryType("points")
        # Create an object set to contain the points and use this set to create the line
        obs = lusas.newObjectSet()                 
        obs.add(self.P1.LusasPoint)
        obs.add(self.P2.LusasPoint)
        # Create the line and save it
        self.LusasLine = obs.createLine(geom_data).getObjects("Line")[0]

In [5]:
toe_length = (base_width-wall_thickness)/2
inner_wall_height = wall_height-beam_thickness

In [6]:
lines = []
lines.append(Line(Point(0,0),   Point(base_width, 0)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(0           , base_thickness)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(-toe_length , 0)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(0           , inner_wall_height)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(+beam_length, 0)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(0           , -inner_wall_height)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(-toe_length , 0)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(0           , -base_thickness)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(base_width  , 0)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(0           , base_thickness)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(-toe_length , 0)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(0           , wall_height)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(-beam_length-2*wall_thickness  , 0)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(0           , -wall_height)))
lines.append(Line(lines[-1].P2, lines[-1].P2.offset(-toe_length , -0)))
lines.append(Line(lines[-1].P2, lines[0].P1)) # Close the lines

In [7]:
line : Line
for line in lines:
    line.create_lusas_line()

In [8]:
geom_data = lusas.geometryData().setAllDefaults()
geom_data.setCreateMethod("planar")
geom_data.setLowerOrderGeometryType("lines")
surface = lusas.newObjectSet().add([l.LusasLine for l in lines]).createSurface(geom_data).getObjects("Surface")[0]


In [9]:
bars20 = []

# Base 1 Cage
bars20.append(Line(lines[0].P1.offset(cover, cover), lines[0].P2.offset(-cover, cover) ))
bars20.append(Line(bars20[-1].P2, lines[1].P2.offset(-cover, -cover) ))
bars20.append(Line(bars20[-1].P2, bars20[-1].P2.offset(-base_width+2*cover, 0) ))
bars20.append(Line(bars20[-1].P2, bars20[0].P1 ))

# Base 2 Cage
bars20.append(Line(lines[8].P1.offset(cover, cover), lines[8].P2.offset(-cover, cover) ))
bars20.append(Line(bars20[-1].P2, lines[9].P2.offset(-cover, -cover) ))
bars20.append(Line(bars20[-1].P2, bars20[-1].P2.offset(-base_width+2*cover, 0) ))
bars20.append(Line(bars20[-1].P2, bars20[4].P1 ))

for bar in bars20:
    bar.create_lusas_line()

In [10]:
bars25 = []
# Left Wall Cage
bars25.append(Line(lines[13].P1.offset(cover, -cover), lines[13].P1.offset(cover, -wall_height - base_thickness + cover) ))
bars25.append(Line(bars25[-1].P2, bars25[-1].P2.offset(wall_thickness-2*cover, 0.0) ))
bars25.append(Line(bars25[-1].P2, bars25[-1].P2.offset(0, base_thickness + wall_height - 2*cover) ))
bars25.append(Line(bars25[-1].P2, bars25[0].P1 ))

# Right Wall Cage
bars25.append(Line(lines[12].P1.offset(-cover, -cover), lines[12].P1.offset(-cover, -wall_height - base_thickness + cover) ))
bars25.append(Line(bars25[-1].P2, bars25[-1].P2.offset(-wall_thickness+2*cover, 0.0) ))
bars25.append(Line(bars25[-1].P2, bars25[-1].P2.offset(0, base_thickness + wall_height - 2*cover) ))
bars25.append(Line(bars25[-1].P2, bars25[-3].P1 ))

# Beam Cage
bars25.append(Line(lines[12].P1.offset(-cover, -cover), lines[12].P1.offset(-beam_length-2*wall_thickness+cover, -cover) ))
bars25.append(Line(bars25[-1].P2, bars25[-1].P2.offset(0, -beam_thickness+2*cover) ))
bars25.append(Line(bars25[-1].P2, bars25[-1].P2.offset(beam_length+2*wall_thickness-2*cover, 0) ))
bars25.append(Line(bars25[-1].P2, bars25[-3].P1 ))

for bar in bars25:
    bar.create_lusas_line()

In [11]:
# Create a plane strain mesh attribute and assign it to the surface
mesh_attr = db.createMeshSurface("Plane Strain Mesh")
mesh_attr.setRegularSize("QPN4M", beam_thickness / 4, True)
mesh_attr.setValue("allowIrregular", True)

# When not passing an assignment object the default assignment settings are used
mesh_attr.assignTo(surface)


# Create parasitic bar element mesh
bar_mesh_attr = db.createMeshLine("Bar Mesh")
bar_mesh_attr.addElementName("BAR2")
bar_mesh_attr.setParasiticType("reinforcement", "allow gaps")
bar_mesh_attr.removeParasiticSpacing()
bar_mesh_attr.setParasiticMaxPercentDeviation(10.0)
bar_mesh_attr.setValue("uiSpacing", "uniform")
# Assign to bar lines
bar_mesh_attr.assignTo([l.LusasLine for l in bars20])  
bar_mesh_attr.assignTo([l.LusasLine for l in bars25])  

# Tell Modeller to generate the mesh
db.resetMesh()
db.updateMesh()


<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFDatabase instance at 0x2278127466448>

In [12]:
# Create a spring support attribute
spring_support_attr = db.createSupportStructural("Sup1")
spring_support_attr.setStructural("S", "S", "F", "F", "F", "F", "F", "F", "C", "F")
spring_support_attr.setSpring("Length", "500.0", "500.0", 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

spring_support_attr.assignTo(lines[0].LusasLine)
spring_support_attr.assignTo(lines[8].LusasLine)

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFObjectSet instance at 0x2278130310256>

In [13]:
# Create reinforcement bar attributes with a helper function
names = Helpers.create_reinforcing_bar_attributes(db, [0.02, 0.025])

stiffness_mod_bars_150 = db.createSectionPropertyModifier("Bars at 150mm")
stiffness_mod_bars_150.setMassFactor("A", 1/0.15)
stiffness_mod_bars_150.setStiffnessFactor("A", 1/0.15)
stiffness_mod_bars_150.setStiffnessFactor("Izz", 1/0.15)
stiffness_mod_bars_150.setStiffnessFactor("Asy", 1/0.15)

# Assign the geometry to the 20mm bars
attr = db.getAttribute("Geometric", names[0])
attr.assignTo([l.LusasLine for l in bars20])

# Assign the geometry to the 25mm bars
attr = db.getAttribute("Geometric", names[1])
attr.assignTo([l.LusasLine for l in bars25])

# All bars are spaced at 150 so increase the bar stiffnesses to reflect their properties per unit width
stiffness_mod_bars_150.assignTo([l.LusasLine for l in bars20])
stiffness_mod_bars_150.assignTo([l.LusasLine for l in bars25])

# Elastic material for the bars
rebar_material_attr = db.createIsotropicMaterial("Reinforcing Steel", 200.0E6, 0.3, 7.85)
rebar_material_attr.assignTo([l.LusasLine for l in bars20])
rebar_material_attr.assignTo([l.LusasLine for l in bars25])


<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFObjectSet instance at 0x2278126863568>

In [14]:
non_linear_concrete = db.createIsotropicMaterial("Nonlinear Concrete", "36.75E6", "0.2", "2.5484")
non_linear_concrete.addPlasticConcrete109("48.0E3", "3.5E3", "2.2E-3", 0.0, "0.13", 
                                          1.15, 0.6, -0.1, 0.425, 0.5, 5.0, 1.25, 0.8, 
                                          0.05, 0.06, 0.03, 0.6, 0.75, 8.0, 8.0, 3.0, 2, 2, 0.0, "fracture")

non_linear_concrete.assignTo(surface)

                                


TypeError: IFMaterialIsotropic.addPlasticConcrete109() takes from 1 to 25 positional arguments but 26 were given

In [301]:
# Show fleshed bars
lusas.view().attributes().visualiseAll("Geometric")

# Loading

In [ ]:
# For a nonlinear analysis we'll assign all the permenent load to the first loadcase and increase the variable load in the second ;loadcase until the structure fails
loadcase_dl = win32.CastTo(db.getLoadset(1), "IFLoadcase")
loadcase_dl.setName("Dead Load")
loadcase_dl.addGravity(True)

nl_control = loadcase_dl.setTransientControl(0).getTransientControl()
# nl_control.setValue("CouplingReadInterval", 1.0).setValue("CouplingWriteInterval", 1.0)
nl_control.setNonlinearManual()
# nl_control.setConstants().setValue("nit", 12).setValue("nalps", 2).setValue("toline", 0.75).setValue("rmaxal", 100.0E6)
# nl_control.setValue("rnoral", 100.0E6).setValue("dlnorm", 0.1).setValue("rlnorm", 0.1).setValue("wlnorm", 100.0E6)
# nl_control.setValue("dtnrml", 0.0).setValue("ampmx", 5.0).setValue("etmxa", 25.0).setValue("etmna", 0.0)
# nl_control.setValue("alpha", 0.0).setValue("beta", 1.0).setValue("gamma", 0.5).setValue("isilcp", False)
# nl_control.setValue("pnrm", 0.1).setValue("ptnrm", 0.0).setValue("tnorm", 0.0).setValue("pnorm", 0.0)
# nl_control.setValue("GroundwaterSolutionType", "Auto").setValue("PermeabilityType", "Auto")
# nl_control.setValue("iterStrategyType", "Auto").setValue("iterDataType", "auto")
nl_control.setOutput()#.setValue("IncrementIntervalForLusas", 1).setValue("IncrementIntervalForPlotFile", 1).setValue("IncrementIntervalForRestart", 0)
# nl_control.setValue("MaxRestartDumpsSaved", 0).setValue("IncrementIntervalForTimeStepLog", 1).setValue("IncrementIntervalForHistory", 1)



In [ ]:
loadcase_ll = db.createLoadcase("Live Load", "Analysis 1", 0, False)

nl_control = loadcase_ll.setTransientControl(0).getTransientControl()

# nl_control.setValue("CouplingReadInterval", 1.0).setValue("CouplingWriteInterval", 1.0)
nl_control.setNonlinearAutomatic(0.1)
nl_control.setValue("Iterations", 4)
nl_control.setValue("MaxChangeInLoadFactor", 0.0)
nl_control.setValue("MaxLoadFactor", 1.0)
nl_control.setValue("IncrementMethod", "Constant load level")
# nl_control.setValue("StiffnessParameter", 0.0).setValue("TerminateOnLimit", false)
# nl_control.setValue("minChangeIncLoad", 1.0E-15).setValue("AllowStepReduction", true)
# nl_control.setValue("ReductionFactor", 0.5).setValue("OriginalLoadIncreaseFactor", 2.0).setValue("MaxStepReduction", 5)
# nl_control.setValue("arcLenPathDir", "By number of negative pivots").setValue("arcLengthUse", "off")


llassignment = lusas.assignment().setAllDefaults().setLoadset(loadcase_ll)

global_dist = db.createLoadingGlobalDistributed("GlbD1")
global_dist.setGlobalDistributed("Length", 0.0, "-100.0", 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, False, 0.0)
global_dist.assignTo(lines[12].LusasLine, llassignment)

# Solve

In [ ]:
db.getAnalysis("Analysis 1").solve(True)
db.openAllResults()

In [ ]:
lusas.view().insertContoursLayer()
lusas.view().contours().setResultsTransformNone()
lusas.view().contours().setResults("Concrete results - Plane Strain", "CWMax")
lusas.view().contours().setDisplayType("ElementNodal")